In [1]:
import numpy as np
import cv2

In [2]:
# 카메라에 연결한다.
# 매개변수에는 카메라 장비 id 값을 넣어준다.
# 노트북에 장착되어 있는 내장카메라 : 0
# usb에 연결 되어있는 카메라는 1 ~ n 까지에 해당한다.
camera = cv2.VideoCapture(0)

In [4]:
# 카메라 사용 가능 여부 확인
camera.isOpened()

False

In [5]:
# 촬영 사이즈
camera.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 640)

False

In [2]:
# CasCade Classifier 생성
face_cascade = cv2.CascadeClassifier()
eyes_cascade = cv2.CascadeClassifier()

In [3]:
# 가중치 데이터가 들어있는 xml 파일명
face_xml = './opencv_data/haarcascades/haarcascade_frontalface_alt.xml'
eyes_xml = './opencv_data/haarcascades/haarcascade_eye_tree_eyeglasses.xml'

In [4]:
# 가중치 데이터를 학습모델에 적용한다.
face_cascade.load(cv2.samples.findFile(face_xml))
eyes_cascade.load(cv2.samples.findFile(eyes_xml))

True

In [ ]:
# 촬영한다.
while cv2.waitKey(33) < 0 :
        
    # 현재 촬영된 사진을 가져온다.
    ret, frame = camera.read()
    
    # 좌우반전
    frame = cv2.flip(frame, 1)
    
    # 인식률 향상을 이미지 데이터 전처리
    # 흑백으로 변환한다.
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 데이터를 표준화한다.
    frame_gray = cv2.equalizeHist(frame_gray)
    
    # 얼굴을 인식한다.
    # 결과는 인식된 부분의 이미지상의 좌측 상단 x, y 좌표와 가로, 세로 길이를
    # 가져온다.
    faces = face_cascade.detectMultiScale(frame_gray)
    
    # 인식한 얼굴의 수 만큼 반복한다.
    for x, y, w, h in faces :
        # 인식한 얼굴에 네모를 그려준다.
        # 이미지, 좌측상단 x, 좌측상단 y, 우측하단 x, 우측하단 y, 색상, 두께
        frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 4)

        # 현재의 얼굴 부분만 데이터로 가져온다.
        faceROI = frame_gray[y:y+h, x:x+h]
        # 추출한 얼굴에서 눈을 가져온다.
        eyes = eyes_cascade.detectMultiScale(faceROI)
        # print(eyes)
        # print('---------------------------------------------')

        # 검출된 눈의 수 만큼 반복한다.
        for x2, y2, w2, h2 in eyes :
            # 눈의 중앙점
            eye_x = x + x2 + w2 // 2
            eye_y = y + y2 + h2 // 2
            # 반경을 계산한다.
            radius = int(round((w2 + h2) * 0.25))
            # 원을 그린다.
            # 그릴이미지, 원의 중앙점 xy 좌표, 반경, 색상, 두께
            frame = cv2.circle(frame, (eye_x, eye_y), radius, (255, 0, 0), 4)
    
    # 그린다.
    cv2.imshow('Camera', frame)
    
# 동영상 데이터를 제거한다.
camera.release()
cv2.destroyAllWindows()